<a href="https://colab.research.google.com/github/mirceaachirili/youtube-summarize/blob/main/Copy_of_youtube_summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install git+https://github.com/openai/whisper.git 
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-jhlp92i0
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-jhlp92i0
  Resolved https://github.com/openai/whisper.git to commit f82bc59f5ea234d4b97fb2860842ed38519f7e65
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 10.7 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175340 sha256=0aa7e71e7b8455b1d5f3b47f6a86511452f43d6f86c632c7455ba5a7074cc2f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-nthlkssl/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper
Looking in index

In [ ]:
# Import packages 
import whisper
import pytube as pt
from transformers import pipeline

In [ ]:
model = whisper.load_model("base.en")

summarizer = pipeline('summarization', model='philschmid/bart-large-cnn-samsum')

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
# Download audio from youtube video
yt = pt.YouTube(input())

stream = yt.streams.filter(only_audio=True)[0]
stream.download(filename="audio_english.mp3")

https://www.youtube.com/watch?v=KbuJY2pqZTk


'/content/audio_english.mp3'

In [ ]:
# Transcribe audio to text
result = model.transcribe("audio_english.mp3")
yt_text = result['text']

/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
# Preview text
yt_text

" Welcome to Thursday 12 January.<eos> There's a new Covid variant around XBB 1.<eos>5.<eos> And basically I've concluded that we're pretty well all of us are going to be infected with it or at least certainly exposed to it in the next few months.<eos> The good news is I'm not particularly worried about it so let's look at it now.<eos> Now if we start off with the American data from the CDC, we see that the BA5 has gradually gone down.<eos> The one we thought was so transmissible originally.<eos> That's been outcomputed by these BQ1 and BQ1 variants of the BQ1 variant and it's sub variant BQ1 one.<eos> But now XB15 is really starting to take off.<eos> And the growth advantage here is about 40% over BQ1 so it's reproducing much more quickly and spreading much more quickly.<eos> And it's going to become the predominant variant and we're all going to be exposed to it.<eos> But let's look at the detail now.<eos> So this is from Dr.<eos> Jia, the White House coordinator, COVID-19 coordinato

In [ ]:
# Replace end of sentence characters
yt_text = yt_text.replace('.', '.<eos>')
yt_text = yt_text.replace('?', '?<eos>')
yt_text = yt_text.replace('!', '!<eos>')


In [ ]:
# Split text into chunks of maximum 500 words 
sentences = yt_text.split('<eos>')

split_limit = 500

current_split = 0
splits = []

for sentence in sentences:
  if len(splits) == current_split + 1:
    
    if len(splits[current_split]) + len(sentence.split(' ')) <= split_limit:
      splits[current_split].extend(sentence.split(' '))
    else:
      current_split += 1
      splits.append(sentence.split(' '))
  
  else:
    print(current_split)
    splits.append(sentence.split(' '))

for split_id in range(len(splits)):
  splits[split_id] = ' '.join(splits[split_id])


0


In [ ]:
# Review split count
len(splits)

7

In [ ]:
# Preview first split
splits[0]

" Welcome to Thursday 12 January.  There's a new Covid variant around XBB 1. 5.  And basically I've concluded that we're pretty well all of us are going to be infected with it or at least certainly exposed to it in the next few months.  The good news is I'm not particularly worried about it so let's look at it now.  Now if we start off with the American data from the CDC, we see that the BA5 has gradually gone down.  The one we thought was so transmissible originally.  That's been outcomputed by these BQ1 and BQ1 variants of the BQ1 variant and it's sub variant BQ1 one.  But now XB15 is really starting to take off.  And the growth advantage here is about 40% over BQ1 so it's reproducing much more quickly and spreading much more quickly.  And it's going to become the predominant variant and we're all going to be exposed to it.  But let's look at the detail now.  So this is from Dr.  Jia, the White House coordinator, COVID-19 coordinator.  And he actually says the spread of XBB 1. 5 is s

In [ ]:
# Summarize text 
summarized = summarizer(splits, max_length=120, min_length=30, do_sample=False)

In [ ]:
# Join chunks of summarized text
text = ' '.join([summary['summary_text'] for summary in summarized])

In [ ]:
# View final summarized text
text

"There is a new Covid variant around XBB 1.5. 80% of Americans who have already been infected with it are likely to catch it again. Dr. Jia, the White House coordinator, COVID-19 coordinator, says the spread of X BB 1. 5 is stunning. XB15 is reproducing much more quickly and spreading more quickly than BA5.  The official guideline from the CDC and from the White House is avoid getting infected. There is a high incidence of COVID in the UK. 4.5% of the population in England have COVID. BQ11 is still the main strain. XBB 1. 5 and BA5 and BA will replace it. BQ5 reduced hospitalization after vaccine by 64% two weeks after the vaccine was given. The protection from BQ1 against hospitalization is down to 50%. The original Wuhan virus that we're vaccinating against no longer exists in the general population. BQ is going to be one of the shortest live variants ever. XpB is a new strain of the common cold virus. It has more mutations than BA2. Most of the symptoms are going to be nasal or phar